In [0]:
dbutils.fs.ls("/mnt/silver/SalesLT")

[FileInfo(path='dbfs:/mnt/silver/SalesLT/Address/', name='Address/', size=0, modificationTime=1736109458000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/Customer/', name='Customer/', size=0, modificationTime=1736109461000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/CustomerAddress/', name='CustomerAddress/', size=0, modificationTime=1736109464000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/Product/', name='Product/', size=0, modificationTime=1736109467000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductCategory/', name='ProductCategory/', size=0, modificationTime=1736109470000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductDescription/', name='ProductDescription/', size=0, modificationTime=1736109472000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductModel/', name='ProductModel/', size=0, modificationTime=1736109475000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductModelProductDescription/', name='ProductModelProductDescription/', size=0, modificationTime=1736109477000),
 FileInf

In [0]:
from pyspark.sql.functions import col

In [0]:
from pyspark.sql.functions import col

def rename_columns_to_snake_case(df):
    """
    Convert column names from PascalCase or camelCase to snake_case in a PySpark DataFrame.

    Args:
        df (DataFrame): The input DataFrame with columns to be renamed.

    Returns:
        DataFrame: A new DataFrame with column names converted to snake_case.
    """
    # Get the list of column names
    column_names = df.columns

    # Dictionary to hold old and new column name mappings
    rename_map = {}

    for old_col_name in column_names:
        # Convert column name from PascalCase or camelCase to snake_case
        new_col_name = "".join([
            "_" + char.lower() if (
                char.isupper()              # Check if the current character is uppercase
                and idx > 0                 # Ensure it's not the first character
                and not old_col_name[idx - 1].isupper()  # Ensure the previous character is not uppercase
            ) else char.lower()  # Convert character to lowercase
            for idx, char in enumerate(old_col_name)
        ]).lstrip("_")  # Remove any leading underscore

        # Avoid renaming to an existing column name
        if new_col_name in rename_map.values():
            raise ValueError(f"Duplicate column name found after renaming: '{new_col_name}'")

        # Map the old column name to the new column name
        rename_map[old_col_name] = new_col_name

    # Rename columns using the mapping
    for old_col_name, new_col_name in rename_map.items():
        df = df.withColumnRenamed(old_col_name, new_col_name)

    return df

In [0]:
table_name = []

for i in dbutils.fs.ls("mnt/silver/SalesLT"):
    table_name.append(i.name.split("/")[0])

table_name

['Address',
 'Customer',
 'CustomerAddress',
 'Product',
 'ProductCategory',
 'ProductDescription',
 'ProductModel',
 'ProductModelProductDescription',
 'SalesOrderDetail',
 'SalesOrderHeader']

In [0]:
dbutils.fs.ls("mnt/silver/SalesLT")

[FileInfo(path='dbfs:/mnt/silver/SalesLT/Address/', name='Address/', size=0, modificationTime=1736109458000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/Customer/', name='Customer/', size=0, modificationTime=1736109461000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/CustomerAddress/', name='CustomerAddress/', size=0, modificationTime=1736109464000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/Product/', name='Product/', size=0, modificationTime=1736109467000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductCategory/', name='ProductCategory/', size=0, modificationTime=1736109470000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductDescription/', name='ProductDescription/', size=0, modificationTime=1736109472000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductModel/', name='ProductModel/', size=0, modificationTime=1736109475000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductModelProductDescription/', name='ProductModelProductDescription/', size=0, modificationTime=1736109477000),
 FileInf

In [0]:
for name  in table_name:
    path = "/mnt/silver/SalesLT/" + name
    print(path)

    df = spark.read.format("delta").load(path)

    df = rename_columns_to_snake_case(df)

    output_path = "/mnt/gold/SalesLT/" + name + "/"
    df.write.format("delta").mode("overwrite").save(output_path)

/mnt/silver/SalesLT/Address
/mnt/silver/SalesLT/Customer
/mnt/silver/SalesLT/CustomerAddress
/mnt/silver/SalesLT/Product
/mnt/silver/SalesLT/ProductCategory
/mnt/silver/SalesLT/ProductDescription
/mnt/silver/SalesLT/ProductModel
/mnt/silver/SalesLT/ProductModelProductDescription
/mnt/silver/SalesLT/SalesOrderDetail
/mnt/silver/SalesLT/SalesOrderHeader


In [0]:
dbutils.fs.ls("/mnt/gold/SalesLT")

[FileInfo(path='dbfs:/mnt/gold/SalesLT/Address/', name='Address/', size=0, modificationTime=1736110609000),
 FileInfo(path='dbfs:/mnt/gold/SalesLT/Customer/', name='Customer/', size=0, modificationTime=1736110612000),
 FileInfo(path='dbfs:/mnt/gold/SalesLT/CustomerAddress/', name='CustomerAddress/', size=0, modificationTime=1736110614000),
 FileInfo(path='dbfs:/mnt/gold/SalesLT/Product/', name='Product/', size=0, modificationTime=1736110616000),
 FileInfo(path='dbfs:/mnt/gold/SalesLT/ProductCategory/', name='ProductCategory/', size=0, modificationTime=1736110618000),
 FileInfo(path='dbfs:/mnt/gold/SalesLT/ProductDescription/', name='ProductDescription/', size=0, modificationTime=1736110620000),
 FileInfo(path='dbfs:/mnt/gold/SalesLT/ProductModel/', name='ProductModel/', size=0, modificationTime=1736110622000),
 FileInfo(path='dbfs:/mnt/gold/SalesLT/ProductModelProductDescription/', name='ProductModelProductDescription/', size=0, modificationTime=1736110624000),
 FileInfo(path='dbfs:/mn